<a href="https://colab.research.google.com/github/Rashika-Gupta/handsonml2/blob/main/cnn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn

In [2]:
image_path = './'
transform = transforms.Compose([transforms.ToTensor()])

In [3]:
mnist_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True,
                                           transform=transform,
                                           download=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 99843929.72it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 25321006.23it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 39990813.98it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 1110170.67it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
from torchvision.datasets import mnist
#creating the dataset into valid and training ds
from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))

#getting the test dataset
mnist_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=False,
                                           transform=transform,
                                           download=False)


In [5]:
#mnist_train_dataset.shape

In [6]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle = True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle = True)


In [7]:
model = nn.Sequential() #stacking different layers - convolution, pooling, drop-out
model.add_module(
    'conv1', nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 5, padding= 2), #defingin kernel_Size = 5*5, first convolution has 32 output features (32 filters )
      )
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module('conv2', nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size= 5, padding = 2))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

The pytorch takes input as the form of NCHW ( # of images in a batch, C for channels, H heigh, W width)

In [8]:
x = torch.ones((4,1,28,28))
model(x) # passing atuple where there are 4 images per batch, 1 cannel (i.e) 2 d image or on gray scale and 28*28 is the pixel length
model(x).shape

torch.Size([4, 64, 7, 7])

the shape taht we got is because the output llayer is 64 (i.e - 64 kernels in the end or 64 filter/feature extraction, 28 reduced to 7 because there is pooling applied twice which has  reducd by 4)

In [9]:
# to feed in the deep nn
#flatten it
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p = 0.5))
model.add_module('fc2', nn.Linear(1024, 10))

In [ ]:
#defning the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
#define the training function
def train (model, num_epochs, train_dl, valid_dl):
  loss_hist_train = [0]*num_epochs
  accuracy_hist_train =[0]*num_epochs
  loss_hist_train = [0]*num_epochs
  accuracy_hist_train =[0]*num_epochs
  for epoch in range(num_epochs):
    model.train()
    for x_batch, y_batch in train_dl:
      pred = model(x_batch)
      loss = loss_fn(pred, y_batch)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      loss_hist_train[epoch] += loss.item()*y_batch.size(0)

      is_correct = ( torch.argmax(pred, dim = 1) == y_batch).float()
      accuracy_hist_train[epoch] += is_correct.sum()

    loss_hist_train[epoch] /= len(train_dl.dataset)
    accuracy_hist_train[epoch] /= len(train_dl.dataset)

    model.eval()


